In [6]:
import os
os.environ['HACK_DATA_PATH'] = '../data/sentiment_dataset'

from ChatGPTRequestManager import RequestManager
import numpy as np
import pandas as pd
import final_solution

In [22]:
generated_dataset = pd.DataFrame(columns=['text', 'entity_idx', 'start_pos', 'end_pos', 'score'])
if not os.path.exists('generated.csv'):
    generated_dataset.to_csv('generated_dataset.csv', index=False)
else:
    generated_dataset = pd.read_csv('generated_dataset.csv')

In [23]:
manager = RequestManager()

def get_scores(text: str, entities: list):
    prompt = text + '\n' + 'Компании: '
    used = set()
    names = {}
    for eid, (start_pos, end_pos) in entities:
        if eid in used:
            continue
        used.add(eid)
        name = text[start_pos:end_pos + 1]
        prompt += name + ', '
        names[name] = eid
        names[eid] = name
        
    prompt = prompt[:-2]
    print(f"Request of length {len(prompt)}")
    response = manager.write_one_message_with_role(prompt).split('\n')
    
    sentiments = {}
    for i in range(len(response)):
        name = response[i].split(':')[0]
        sentiment = response[i].split(':')[1]
        
        try:
            eid = names[name.strip()]
            sentiments[eid] = int(sentiment.strip())
        except:
            continue
            
    return [sentiments.get(eid, 0) for eid, (_, _) in entities]
        
        
get_scores('Apple упал', [(1, (0, 4))])
        

Request of length 26


[2]

In [27]:
from tqdm import tqdm

targets = ['markettwits.csv', 'smartlabnews.csv', 'MarketOverview.csv']

for filename in os.listdir('../download/data_large'):
    if filename not in targets:
        continue
    if filename.endswith('.csv'):
        print(f'Processing {filename}')
        df = pd.read_csv(f'../download/data_large/{filename}')
        if df.shape[0] > 1000:
            df = df.sample(1000)
    else:
        continue
        
    texts = df['text'].dropna().values.tolist()
    all_entities = final_solution.baseline.find_entities(texts)
    
    for text, entities in tqdm(zip(texts, all_entities)):
        if len(entities) == 0:
            continue
        
        scores = get_scores(text, entities)
        print(scores)
        
        for i, (eid, (start_pos, end_pos)) in enumerate(entities):
            if scores[i] not in [1, 2, 3, 4, 5]:
                continue
            generated_dataset = pd.concat([
                generated_dataset,
                pd.DataFrame([[text, eid, start_pos, end_pos, scores[i]]], columns=['text', 'entity_idx', 'start_pos', 'end_pos', 'score'])
            ])
            generated_dataset.to_csv('generated_dataset.csv', index=False)
                    
    

Processing smartlabnews.csv


100%|██████████| 945/945 [00:20<00:00, 45.38it/s]
0it [00:00, ?it/s]

Request of length 3845
[3, 3, 1, 4, 4, 4, 5, 5, 1, 1]
Added
Added
Added
Added
Added
Added
Added


1it [00:02,  2.13s/it]

Added
Added
Added
Request of length 2386
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Added
Added
Added
Added
Added
Added
Added
Added
Added
Added
Added
Added
Added
Added
Added


...

[2]
Added
Request of length 2789


407it [06:58,  1.14it/s]

[0]
Request of length 3027


407it [06:59,  1.03s/it]


KeyboardInterrupt: 